## Defining the Dataset

In this dataset we will be detecting 3 types of objects: Vehicles, People and animals. The structure of the dataset is as below.

1. A numpy array of all the RGB Images (3x300x400)

2. A numpy array of all the masks (300x400)

3. List of ground truth labels per image

4. List of ground truth bounding box per image. The four numbers are the upper left and lower right coordinates

In [1]:
import os
import cv2
import argparse
from PIL import Image
import h5py
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import scipy 
%matplotlib inline

# Created the Class for the custom dataset
class CustomDataset(torch.utils.data.Dataset):
    
    def __init__(self, root_img, root_mask, root_npy_labels, root_npy_bboxes, transforms = None):
        
        """
        Inputs:
        root_img: The path to the root directory where the image .h5 files are stored
        root_mask: The path to the root directory where the mask .h5 files are stored
        root_npy_labels: The path to the .npy dataset for labels
        root_npy_bboxes: The path to the .npy dataset for the ground truth bounding boxes
        transforms: Apply a Pytorch transform to each instance of the image
        
        """
        self.root_img = root_img
        self.root_mask = root_mask
        self.root_npy_labels = root_npy_labels
        self.root_npy_bboxes = root_npy_bboxes
        self.transforms = transforms 
        
        self.imgs = h5py.File(self.root_img, 'r')
        self.mask = h5py.File(self.root_mask, 'r')
        self.labels = np.load(self.root_npy_labels, allow_pickle = True)
        self.bboxes = np.load(self.root_npy_bboxes, allow_pickle = True)
        
    # To support indexing when an object of the CustomDataset Class is created
    def __getitem__(self, index):
        
        # Convert the Masks and the input image into an array
        image = np.array(self.imgs['data']).astype('int32')
        masks = np.array(self.mask['data']).astype('int32')
        
        # Convert the Mask, image, bounding boxes and labels to a Pytorch Tensor
        image = torch.as_tensor(image[index])
        masks = torch.as_tensor(masks[index])
        bounding_boxes = torch.as_tensor(self.bboxes[index])
        labels = torch.as_tensor(self.labels[index])
        
        batch = {} 
        batch["bounding_boxes"] = bounding_boxes
        batch["masks"] = masks
        batch["labels"] = labels
        
        if self.transforms is not None:
            image, batch = self.transforms(image,batch)
        return image, batch

In [2]:
root1 = 'C:\\Users\\shant\\Mask_RCNN_Segmentation\\dataset\\mycocodata_img_comp_zlib.h5'
root2 = 'C:\\Users\\shant\\Mask_RCNN_Segmentation\\dataset\\mycocodata_mask_comp_zlib.h5'
root3_npy = 'C:\\Users\\shant\\Mask_RCNN_Segmentation\\dataset\\mycocodata_labels_comp_zlib.npy'
root4_npy = 'C://Users//shant//Mask_RCNN_Segmentation//dataset/mycocodata_bboxes_comp_zlib.npy'

dataset = CustomDataset(root1, root2, root3_npy, root4_npy)
dataset[12]

(tensor([[[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]],
 
         [[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]],
 
         [[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]]], dtype=torch.int32),
 {'bounding_boxes': tensor([[100.0000, 120.0094, 207.8875, 268.9359],
          [183.9109,  85.9969, 263.7531, 274.3828]]),
  'masks': tensor([[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [0, 0, 0,  ..

In [8]:
root1 = 'C:\\Users\\shant\\Mask_RCNN_Segmentation\\dataset\\mycocodata_img_comp_zlib.h5'
root2 = 'C:\\Users\\shant\\Mask_RCNN_Segmentation\\dataset\\mycocodata_mask_comp_zlib.h5'
img = h5py.File(root1,'r')
# You can Inspect what is inside the dataset by using the command list(x.keys())
imgs = np.array(img['data']).astype('int32')
mask = h5py.File(root2,'r')
torch.as_tensor(imgs[0])
#masks = np.array(mask['data'])
#print(f'Number of images: {imgs.shape} Number of Mask: {masks.shape}')
labels = np.load('C:\\Users\\shant\\Mask_RCNN_Segmentation\\dataset\\mycocodata_labels_comp_zlib.npy', allow_pickle=True)
bounding_box = np.load('C:\\Users\\shant\\Mask_RCNN_Segmentation\\dataset\\mycocodata_bboxes_comp_zlib.npy', allow_pickle = True)
#torch.as_tensor(labels[0])
imgs

AttributeError: 'str' object has no attribute 'shape'